# 코드 실습에 앞서 머신러닝, 딥러닝에 대해 학습

## 예: 지도학습, 비지도학습, 군집화, 신경망 개념, 손실 함수, 최적화(경사하강법, 역전파, 학습률), 최적화 알고리즘(SGD, Adam)

## 텐서란?
### 텐서는 numpy에 존재하는데 배열, 행렬과 유사한 자료 구조 

In [1]:
import torch

print(torch.tensor([1, 2, 3])) # 입력 데이터 형식을 참조해 해당 자료형 텐서를생성, 입력 데이터 복사해서 생성
print(torch.Tensor([[1, 2, 3],[4, 5, 6]])) # 자동으로 float 형식의 텐서 생성, # 텐서 인스턴스 생성
print(torch.LongTensor([1, 2, 3])) # Long 자료형 텐서
print(torch.FloatTensor([1, 2, 3])) # float형 자료형 텐서 

tensor([1, 2, 3])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([1, 2, 3])
tensor([1., 2., 3.])


In [2]:
import torch


tensor = torch.rand(1, 2)
print(tensor) 
print(tensor.shape) # 텐서 차원 출력
print(tensor.dtype) # 텐서 타입 출력
print(tensor.device) # 텐서의 저장 위치

tensor([[0.9081, 0.7480]])
torch.Size([1, 2])
torch.float32
cpu


In [3]:
import torch

tensor = torch.rand(1, 2)
print(tensor)
print(tensor.shape)
tensor = tensor.reshape(2, 1) #차원 변환 # view()도 존재한다.
print(tensor)
print(tensor.shape)

tensor([[0.0529, 0.3562]])
torch.Size([1, 2])
tensor([[0.0529],
        [0.3562]])
torch.Size([2, 1])


In [4]:
import torch


tensor = torch.rand((3, 3), dtype=torch.float) # 자료형 설정 방법
print(tensor)

tensor([[0.3186, 0.6584, 0.0475],
        [0.2441, 0.4476, 0.7344],
        [0.1849, 0.3153, 0.3337]])


In [5]:
import torch


device = "cuda" if torch.cuda.is_available() else "cpu" 
cpu = torch.FloatTensor([1, 2, 3])
gpu = torch.cuda.FloatTensor([1, 2, 3])
tensor = torch.rand((1, 1), device=device) # 텐서의 저장 위치 설정
print(device)
print(cpu)
print(gpu)
print(tensor)

cuda
tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')
tensor([[0.0758]], device='cuda:0')


C:\Users\user\AppData\Local\Temp\ipykernel_40620\721341711.py:6: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:85.)
  gpu = torch.cuda.FloatTensor([1, 2, 3])


In [6]:
# 애플 실리콘이 탑재된 맥 사용자의 경우
import torch


device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu"
cpu = torch.FloatTensor([1, 2, 3])
mps = torch.rand((1, 1), device=device)
print(device)
print(mps)

cpu
tensor([[0.5972]])


In [7]:
import torch


cpu = torch.FloatTensor([1, 2, 3])
gpu = cpu.cuda() # gpu()로 변경
gpu2cpu = gpu.cpu() # Cpu()로 변경
cpu2gpu = cpu.to("cuda") # to('장치')로도 가능
print(cpu)
print(gpu)
print(gpu2cpu)
print(cpu2gpu)

tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')
tensor([1., 2., 3.])
tensor([1., 2., 3.], device='cuda:0')


In [8]:
import torch
import numpy as np

"""
numpy 배열을 파이토치 텐서로
"""
ndarray = np.array([1, 2, 3], dtype=np.uint8) 
print(torch.tensor(ndarray)) 
print(torch.Tensor(ndarray))
print(torch.from_numpy(ndarray))

tensor([1, 2, 3], dtype=torch.uint8)
tensor([1., 2., 3.])
tensor([1, 2, 3], dtype=torch.uint8)


In [9]:
import torch

"""
detach()는 파이토치 텐서의 경우 모든 연산을 추적하여 역전파 연산등에 사용됨으로 
연산 그래프에서 제거하기 위해 사용해야함
"""
tensor = torch.cuda.FloatTensor([1, 2, 3])
ndarray = tensor.detach().cpu().numpy() #파이토치 텐서 자료형을 numpy 배열로
print(ndarray)
print(type(ndarray))

import torch


#tensor = torch.FloatTensor([1, 2, 3]).to("mps") #mac 에서는 주석 해제 해서 실행
ndarray = tensor.detach().cpu().numpy() 
print(ndarray)
print(type(ndarray))

[1. 2. 3.]
<class 'numpy.ndarray'>
[1. 2. 3.]
<class 'numpy.ndarray'>


## 단순 선형 회귀 구현
### Numpy

In [10]:
import numpy as np
# 입력
x = np.array(
    [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]]
) 
# 정답
y = np.array(
    [[0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]]
)

In [11]:
weight = 0.0 # 초기 가중치 
bias = 0.0 # 초기 bias
learning_rate = 0.005 # 학습률

In [12]:
for epoch in range(10000): #epoch 학습 횟수
    y_hat = weight * x + bias # 정답 예측 선형 회귀 이므로 y=wx+b

    cost = ((y - y_hat) ** 2).mean() # 오차 측정 mse 

    weight = weight - learning_rate * ((y_hat - y) * x).mean() # 가중치 조절
    bias = bias - learning_rate * (y_hat - y).mean() # bias 조절

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Weight : {weight:.3f}, Bias : {bias:.3f}, Cost : {cost:.3f}")

Epoch : 1000, Weight : 0.872, Bias : -0.290, Cost : 1.377
Epoch : 2000, Weight : 0.877, Bias : -0.391, Cost : 1.373
Epoch : 3000, Weight : 0.878, Bias : -0.422, Cost : 1.372
Epoch : 4000, Weight : 0.879, Bias : -0.432, Cost : 1.372
Epoch : 5000, Weight : 0.879, Bias : -0.435, Cost : 1.372
Epoch : 6000, Weight : 0.879, Bias : -0.436, Cost : 1.372
Epoch : 7000, Weight : 0.879, Bias : -0.436, Cost : 1.372
Epoch : 8000, Weight : 0.879, Bias : -0.436, Cost : 1.372
Epoch : 9000, Weight : 0.879, Bias : -0.436, Cost : 1.372
Epoch : 10000, Weight : 0.879, Bias : -0.436, Cost : 1.372


### Pytorch 선형 회귀

In [13]:
from torch import optim

In [14]:
x = torch.FloatTensor([
    [1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]
])
y = torch.FloatTensor([
    [0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]
])

In [15]:
weight = torch.zeros(1, requires_grad=True)
bias = torch.zeros(1, requires_grad=True)
learning_rate = 0.001

In [16]:
optimizer = optim.SGD([weight, bias], lr=learning_rate) # SGD 옵티마이저 사용

In [17]:
for epoch in range(10000):
    hypothesis = weight * x + bias
    cost = torch.mean((hypothesis - y) ** 2)

    optimizer.zero_grad() # 이전 gradients 값을 0으로 설정, 텐서의 기울기는 grad 속성에 누적되어 더해지기 때문에, 
    cost.backward() # loss 역전파
    optimizer.step() # #최적화 함수에 계산된 가중치와 편향들을 반영

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Weight : {weight.item():.3f}, Bias : {bias.item():.3f}, Cost : {cost:.3f}")

Epoch : 1000, Weight : 0.864, Bias : -0.138, Cost : 1.393
Epoch : 2000, Weight : 0.870, Bias : -0.251, Cost : 1.380
Epoch : 3000, Weight : 0.873, Bias : -0.321, Cost : 1.375
Epoch : 4000, Weight : 0.875, Bias : -0.364, Cost : 1.373
Epoch : 5000, Weight : 0.877, Bias : -0.391, Cost : 1.373
Epoch : 6000, Weight : 0.878, Bias : -0.408, Cost : 1.372
Epoch : 7000, Weight : 0.878, Bias : -0.419, Cost : 1.372
Epoch : 8000, Weight : 0.878, Bias : -0.425, Cost : 1.372
Epoch : 9000, Weight : 0.879, Bias : -0.429, Cost : 1.372
Epoch : 10000, Weight : 0.879, Bias : -0.432, Cost : 1.372


In [18]:
import torch
from torch import optim


x = torch.FloatTensor([
    [1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]
])
y = torch.FloatTensor([
    [0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]
])

weight = torch.zeros(1, requires_grad=True)
bias = torch.zeros(1, requires_grad=True)
learning_rate = 0.001

optimizer = optim.SGD([weight, bias], lr=learning_rate)

for epoch in range(10000):
    hypothesis = weight * x + bias
    cost = torch.mean((hypothesis - y) ** 2)
    
    print(f"Epoch : {epoch+1:4d}")
    print(f"Step [1] : Gradient : {weight.grad}, Weight : {weight.item():.5f}")

    optimizer.zero_grad()
    print(f"Step [2] : Gradient : {weight.grad}, Weight : {weight.item():.5f}")

    cost.backward()
    print(f"Step [3] : Gradient : {weight.grad}, Weight : {weight.item():.5f}")

    optimizer.step()
    print(f"Step [4] : Gradient : {weight.grad}, Weight : {weight.item():.5f}")
    
    if epoch == 3:
        break

Epoch :    1
Step [1] : Gradient : None, Weight : 0.00000
Step [2] : Gradient : None, Weight : 0.00000
Step [3] : Gradient : tensor([-540.4854]), Weight : 0.00000
Step [4] : Gradient : tensor([-540.4854]), Weight : 0.54049
Epoch :    2
Step [1] : Gradient : tensor([-540.4854]), Weight : 0.54049
Step [2] : Gradient : None, Weight : 0.54049
Step [3] : Gradient : tensor([-198.9818]), Weight : 0.54049
Step [4] : Gradient : tensor([-198.9818]), Weight : 0.73947
Epoch :    3
Step [1] : Gradient : tensor([-198.9818]), Weight : 0.73947
Step [2] : Gradient : None, Weight : 0.73947
Step [3] : Gradient : tensor([-73.2604]), Weight : 0.73947
Step [4] : Gradient : tensor([-73.2604]), Weight : 0.81273
Epoch :    4
Step [1] : Gradient : tensor([-73.2604]), Weight : 0.81273
Step [2] : Gradient : None, Weight : 0.81273
Step [3] : Gradient : tensor([-26.9772]), Weight : 0.81273
Step [4] : Gradient : tensor([-26.9772]), Weight : 0.83970


### torch.nn 신경망 패키지 사용

In [19]:
import torch
from torch import nn
from torch import optim

In [20]:
x = torch.FloatTensor([
    [1], [2], [3], [4], [5], [6], [7], [8], [9], [10],
    [11], [12], [13], [14], [15], [16], [17], [18], [19], [20],
    [21], [22], [23], [24], [25], [26], [27], [28], [29], [30]
])
y = torch.FloatTensor([
    [0.94], [1.98], [2.88], [3.92], [3.96], [4.55], [5.64], [6.3], [7.44], [9.1],
    [8.46], [9.5], [10.67], [11.16], [14], [11.83], [14.4], [14.25], [16.2], [16.32],
    [17.46], [19.8], [18], [21.34], [22], [22.5], [24.57], [26.04], [21.6], [28.8]
])

In [21]:
# torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
"""
in_features: 신경망의 입력 뉴런 수
out_features: 신경망의 출력 뉴런수
"""
layer=nn.Linear(1, 1, bias=True, device=None, dtype=None)
layer

Linear(in_features=1, out_features=1, bias=True)

In [22]:
model = nn.Linear(1, 1)
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [23]:
for epoch in range(10000):
    output = model(x)
    cost = criterion(output, y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[0.8702]], requires_grad=True), Parameter containing:
tensor([-0.2588], requires_grad=True)], Cost : 1.380
Epoch : 2000, Model : [Parameter containing:
tensor([[0.8735]], requires_grad=True), Parameter containing:
tensor([-0.3257], requires_grad=True)], Cost : 1.375
Epoch : 3000, Model : [Parameter containing:
tensor([[0.8755]], requires_grad=True), Parameter containing:
tensor([-0.3673], requires_grad=True)], Cost : 1.373
Epoch : 4000, Model : [Parameter containing:
tensor([[0.8768]], requires_grad=True), Parameter containing:
tensor([-0.3933], requires_grad=True)], Cost : 1.373
Epoch : 5000, Model : [Parameter containing:
tensor([[0.8776]], requires_grad=True), Parameter containing:
tensor([-0.4094], requires_grad=True)], Cost : 1.372
Epoch : 6000, Model : [Parameter containing:
tensor([[0.8781]], requires_grad=True), Parameter containing:
tensor([-0.4194], requires_grad=True)], Cost : 1.372
Epoch : 7000, Model : [Parameter contain

## 데이터세트, 데이터 로더

### 데이터셋은 특정 입력데이터와 정답지를 제공해주기 위해 제공하는 클래스
### 데이터로더는 데이터세트에 저장된 데이터를 어떠한 방식으로 불러와 활용하지는 정의

In [72]:
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd

In [63]:
class CustomDataset(Dataset):
    def __init__(self, data_path):
        """
            입력 데이터의 전처리 과정 수행 메소드
            학습에 필요한 데이터 형태로 변형하는 과정
        """
        self.data = pd.read_csv(data_path)
        
       
        self.X = self.data.iloc[:, 1:-1].values  
        self.y = self.data.iloc[:, -1].values  

        self.label_map = {label: idx for idx, label in enumerate(sorted(set(self.y)))}
        self.y = [self.label_map[label] for label in self.y]

    def __len__(self):
        """
            데이터 전체 개수 반환
        """
        return len(self.data)
    
    def __getitem__(self, idx):
        """
            학습이 진행 될 때, 하나의 데이터 샘플을 반환하는 메소드
        """
        x = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.long)
        return x, y


In [66]:
dataset = CustomDataset('Iris.csv')
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=0, drop_last=False) 
# drop_last: 배치 단위로 데이터를 가져오는 경우 배치보다 작은 경우 존재한다. 이 때, True면 배치 크기보다 작으면 가져오지 않는다

In [71]:
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [70]:
next(iter(dataloader))

[tensor([[7.3000, 2.9000, 6.3000, 1.8000],
         [7.9000, 3.8000, 6.4000, 2.0000],
         [4.8000, 3.4000, 1.9000, 0.2000],
         [6.8000, 2.8000, 4.8000, 1.4000]]),
 tensor([2, 2, 0, 1])]

In [82]:

dataset_size = len(dataset)
train_size = int(dataset_size * 0.7)
validation_size = int(dataset_size * 0.15)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size]) # 데이터셋 분리
print(f"Training Data Size : {len(train_dataset)}")
print(f"Validation Data Size : {len(validation_dataset)}")
print(f"Testing Data Size : {len(test_dataset)}")

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

Training Data Size : 105
Validation Data Size : 22
Testing Data Size : 23


## 모델 구현
### nn.Module 상속 받아 서브클래스로 제작

In [91]:
import torch.nn.functional as F

In [122]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        """
            모델 레이어 선언
        """
        self.layer1 = nn.Linear(4, 4)
        self.layer2 = nn.Linear(4, 3)
    def forward(self, x):
        """
            순전파 과정 작성
        """
        out = F.relu(self.layer1(x))
        out = F.relu(self.layer2(out))
        
        return out
        

In [124]:
model = CustomModel().to(device)
optimizer = optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

## iris dataset train and model save

### 모델 저장 방법은 torch.save(model, path), 불러오는 방법 torch.load(path, map_location) map_location: 모델 장치 설정 

### 중요한점은 model 인스턴스를 저장 할 때, 모델 전체를 저장 할 수 도 있고, 모델 상태만 저장 가능

In [125]:
for epoch in range(10000):
    train_loss = 0
    val_loss = 0
    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss += loss.item()
    
    print(f'{epoch}, train_loss: {train_loss/len(train_dataloader)}, val_loss: {val_loss/len(validation_dataloader)}')
       

0, train_loss: 1.0970689058303833, val_loss: 1.1208529949188233
1, train_loss: 1.0964369361217206, val_loss: 1.144818162918091
2, train_loss: 1.0926966735949883, val_loss: 1.0866971015930176
3, train_loss: 1.0899724524754744, val_loss: 1.126435399055481
4, train_loss: 1.0946032519523914, val_loss: 1.1120399236679077
5, train_loss: 1.0940483625118549, val_loss: 1.095466959476471
6, train_loss: 1.0906881873424237, val_loss: 1.1235671520233155
7, train_loss: 1.088030111331206, val_loss: 1.1226458311080934
8, train_loss: 1.086980938911438, val_loss: 1.1076229810714722
9, train_loss: 1.0924096015783458, val_loss: 1.12067391872406
10, train_loss: 1.0868570827520811, val_loss: 1.0908809542655944
11, train_loss: 1.0867642783201659, val_loss: 1.0914372205734253
12, train_loss: 1.0863305101027856, val_loss: 1.1200945854187012
13, train_loss: 1.0909379835312183, val_loss: 1.1174624919891358
14, train_loss: 1.0854188272586236, val_loss: 1.0883784532546996
15, train_loss: 1.0846409522570097, val_lo

133, train_loss: 1.074704484297679, val_loss: 1.0692167043685914
134, train_loss: 1.0712237472717578, val_loss: 1.0684911489486695
135, train_loss: 1.0750095156522899, val_loss: 1.0665980100631713
136, train_loss: 1.0745398769011865, val_loss: 1.047727108001709
137, train_loss: 1.0749351909527411, val_loss: 1.063467538356781
138, train_loss: 1.0748280814060798, val_loss: 1.044848084449768
139, train_loss: 1.0708649112628057, val_loss: 1.0468290090560912
140, train_loss: 1.074684787255067, val_loss: 1.0666366457939147
141, train_loss: 1.0747948907888853, val_loss: 1.0658403754234314
142, train_loss: 1.0745175618391771, val_loss: 1.0639983534812927
143, train_loss: 1.0744654627946706, val_loss: 1.0660336017608643
144, train_loss: 1.0735501738694997, val_loss: 1.0618476510047912
145, train_loss: 1.0743083839233105, val_loss: 1.0260773420333862
146, train_loss: 1.070622613796821, val_loss: 1.0254459142684937
147, train_loss: 1.07378024321336, val_loss: 1.061331284046173
148, train_loss: 1.

265, train_loss: 1.06462339254526, val_loss: 1.030836319923401
266, train_loss: 1.0610712927121382, val_loss: 1.0285249471664428
267, train_loss: 1.065127581357956, val_loss: 1.0455490589141845
268, train_loss: 1.0643320633814886, val_loss: 1.0450671792030335
269, train_loss: 1.0607281304322755, val_loss: 1.0258431434631348
270, train_loss: 1.0642430851092706, val_loss: 1.0258256554603578
271, train_loss: 1.064009625178117, val_loss: 1.0490086078643799
272, train_loss: 1.060555806526771, val_loss: 1.027996826171875
273, train_loss: 1.0649529695510864, val_loss: 1.0057157754898072
274, train_loss: 1.0603259526766264, val_loss: 1.0234151244163514
275, train_loss: 1.063694036923922, val_loss: 1.0443276643753052
276, train_loss: 1.0599890970266783, val_loss: 1.025149655342102
277, train_loss: 1.0638133241580083, val_loss: 1.0237481594085693
278, train_loss: 1.0599540770053864, val_loss: 1.0229647517204286
279, train_loss: 1.063552384193127, val_loss: 1.024044644832611
280, train_loss: 1.05

394, train_loss: 1.0503597809718206, val_loss: 1.007102632522583
395, train_loss: 1.0470328330993652, val_loss: 1.0250148177146912
396, train_loss: 1.050133622609652, val_loss: 0.9904473781585693
397, train_loss: 1.0501064199667711, val_loss: 1.0232597589492798
398, train_loss: 1.0499469477396746, val_loss: 1.0288962721824646
399, train_loss: 1.0467208692660699, val_loss: 1.0260076880455018
400, train_loss: 1.0504874197336345, val_loss: 1.003869080543518
401, train_loss: 1.0496044434033907, val_loss: 1.0073952674865723
402, train_loss: 1.0459284140513494, val_loss: 1.0283995747566224
403, train_loss: 1.0500590686614697, val_loss: 1.0256159424781799
404, train_loss: 1.0490489441614885, val_loss: 1.0278268933296204
405, train_loss: 1.0496612328749437, val_loss: 1.0074254989624023
406, train_loss: 1.0460185775390038, val_loss: 1.0025999665260314
407, train_loss: 1.0494046807289124, val_loss: 1.0072341680526733
408, train_loss: 1.0485616899453676, val_loss: 1.0277634978294372
409, train_lo

524, train_loss: 1.0299253440820253, val_loss: 1.0069980025291443
525, train_loss: 1.0297509065041175, val_loss: 1.0045935750007629
526, train_loss: 1.0265552126444304, val_loss: 1.0110839486122132
527, train_loss: 1.0303335052270155, val_loss: 1.0089480400085449
528, train_loss: 1.0296304432245402, val_loss: 0.9913602709770203
529, train_loss: 1.0302438690112188, val_loss: 1.0070276618003846
530, train_loss: 1.0287285309571486, val_loss: 1.0031612396240235
531, train_loss: 1.0292937893133898, val_loss: 0.9859187006950378
532, train_loss: 1.0283800822037916, val_loss: 0.9933483719825744
533, train_loss: 1.0294877015627348, val_loss: 0.9836754918098449
534, train_loss: 1.0289685336443095, val_loss: 1.0005114316940307
535, train_loss: 1.0277900925049415, val_loss: 1.002284300327301
536, train_loss: 1.0284719260839315, val_loss: 0.9909204244613647
537, train_loss: 1.0286233608539288, val_loss: 0.9837003827095032
538, train_loss: 1.0279611440805287, val_loss: 0.9873885989189148
539, train_

...

9416, train_loss: 0.42670338314313155, val_loss: 0.4238878309726715
9417, train_loss: 0.42192023534041184, val_loss: 0.4022571682929993
9418, train_loss: 0.42664637244664705, val_loss: 0.4238292634487152
9419, train_loss: 0.4266389264510228, val_loss: 0.41259989738464353
9420, train_loss: 0.4266210381801312, val_loss: 0.42547120451927184
9421, train_loss: 0.4227487238553854, val_loss: 0.40080217719078065
9422, train_loss: 0.42657982271451217, val_loss: 0.40294389128685
9423, train_loss: 0.42655473890212864, val_loss: 0.4199027240276337
9424, train_loss: 0.4224614432224861, val_loss: 0.41698487997055056
9425, train_loss: 0.423278764463388, val_loss: 0.4197618365287781
9426, train_loss: 0.4236111319982089, val_loss: 0.419939911365509
9427, train_loss: 0.4265114344083346, val_loss: 0.4059534966945648
9428, train_loss: 0.4222931368992879, val_loss: 0.4230912268161774
9429, train_loss: 0.42646819811600906, val_loss: 0.40491386950016023
9430, train_loss: 0.42281207041098523, val_loss: 0.4057

9538, train_loss: 0.42401607506550276, val_loss: 0.4047849714756012
9539, train_loss: 0.4240019539227852, val_loss: 0.4098590135574341
9540, train_loss: 0.4239666439019717, val_loss: 0.412098354101181
9541, train_loss: 0.4210257518749971, val_loss: 0.4124466896057129
9542, train_loss: 0.4209580375598027, val_loss: 0.39871516823768616
9543, train_loss: 0.41998661080232036, val_loss: 0.43541294932365415
9544, train_loss: 0.4191583807651813, val_loss: 0.4229017078876495
9545, train_loss: 0.42015179934409946, val_loss: 0.43064951300621035
9546, train_loss: 0.42094850884034085, val_loss: 0.42015387415885924
9547, train_loss: 0.42381483774918777, val_loss: 0.4062208294868469
9548, train_loss: 0.4190468409886727, val_loss: 0.40356179475784304
9549, train_loss: 0.4237873267668944, val_loss: 0.42560170888900756
9550, train_loss: 0.4237909706739279, val_loss: 0.40207119584083556
9551, train_loss: 0.4196013475839908, val_loss: 0.4306182265281677
9552, train_loss: 0.4211857198522641, val_loss: 0.3

9659, train_loss: 0.41805923214325535, val_loss: 0.41276814937591555
9660, train_loss: 0.41749381445921385, val_loss: 0.3961048603057861
9661, train_loss: 0.4173214945655603, val_loss: 0.41530507802963257
9662, train_loss: 0.416671661230234, val_loss: 0.4183735430240631
9663, train_loss: 0.419223207120712, val_loss: 0.42020665407180785
9664, train_loss: 0.41844628006219864, val_loss: 0.4151999056339264
9665, train_loss: 0.42132206433094466, val_loss: 0.4189599335193634
9666, train_loss: 0.41850519294922167, val_loss: 0.4144638180732727
9667, train_loss: 0.41787666655503786, val_loss: 0.4101219713687897
9668, train_loss: 0.4175957968601814, val_loss: 0.4216961026191711
9669, train_loss: 0.41765973888910735, val_loss: 0.3941351920366287
9670, train_loss: 0.41777320550038266, val_loss: 0.414204353094101
9671, train_loss: 0.41598803435380644, val_loss: 0.42119215726852416
9672, train_loss: 0.42117461676780993, val_loss: 0.40895277857780454
9673, train_loss: 0.4212177023291588, val_loss: 0.

9786, train_loss: 0.4162090844832934, val_loss: 0.39720349907875063
9787, train_loss: 0.41851449471253616, val_loss: 0.40327935218811034
9788, train_loss: 0.41508765117480206, val_loss: 0.4135324597358704
9789, train_loss: 0.4145811286110144, val_loss: 0.41648560762405396
9790, train_loss: 0.4185020396342644, val_loss: 0.41251835227012634
9791, train_loss: 0.4146927171028577, val_loss: 0.4072795808315277
9792, train_loss: 0.41419890981454116, val_loss: 0.4129007697105408
9793, train_loss: 0.4184993660220733, val_loss: 0.41173062920570375
9794, train_loss: 0.414939335332467, val_loss: 0.4104363977909088
9795, train_loss: 0.41495328969680345, val_loss: 0.4189138948917389
9796, train_loss: 0.41843662468286663, val_loss: 0.411615127325058
9797, train_loss: 0.4137138712864656, val_loss: 0.39884900450706484
9798, train_loss: 0.412837137396519, val_loss: 0.41744624376296996
9799, train_loss: 0.41547751082823825, val_loss: 0.4162036538124084
9800, train_loss: 0.4183600281293576, val_loss: 0.40

9915, train_loss: 0.4107331931591034, val_loss: 0.3928119122982025
9916, train_loss: 0.4123136974298037, val_loss: 0.3904542922973633
9917, train_loss: 0.4158860141268143, val_loss: 0.3982112020254135
9918, train_loss: 0.4121719610232573, val_loss: 0.4011370360851288
9919, train_loss: 0.4136640509733787, val_loss: 0.4025693893432617
9920, train_loss: 0.4118945552752568, val_loss: 0.40975362062454224
9921, train_loss: 0.4157940229544273, val_loss: 0.3946944892406464
9922, train_loss: 0.41577546527752507, val_loss: 0.40264729261398313
9923, train_loss: 0.41575488505455166, val_loss: 0.41174728870391847
9924, train_loss: 0.4120400072290347, val_loss: 0.396809446811676
9925, train_loss: 0.41572383504647475, val_loss: 0.40975645184516907
9926, train_loss: 0.4126346609913386, val_loss: 0.40975395441055296
9927, train_loss: 0.41567088892826665, val_loss: 0.41636934876441956
9928, train_loss: 0.4111183898953291, val_loss: 0.41270496547222135
9929, train_loss: 0.4156688285561708, val_loss: 0.42

In [127]:
torch.save(
    model,
    "../models/model.pt"
)

RuntimeError: Parent directory ../models does not exist.

In [ ]:
torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = torch.load("../models/model.pt", map_location=device)
print(model)

In [ ]:
"""
    체크 포인트는 저장은 학습 중 오류 발생등으로 학습이 종료되었을 때, 학습을 처음부터 다시 시작하는 것이 아닌
    저장된 체크 포인트도 학습을 시작하기 위해서 저장함
"""
#torch.save(
#             {
#                 "model": "CustomModel",
#                 "epoch": epoch,
#                 "model_state_dict": model.state_dict(),
#                 "optimizer_state_dict": optimizer.state_dict(),
#                 "cost": cost,
#                 "description": f"CustomModel 체크포인트-{checkpoint}",
#             },
#             f"../models/checkpoint-{checkpoint}.pt",
#         )

In [ ]:

# checkpoint = torch.load("../models/checkpoint-6.pt")
# model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# checkpoint_epoch = checkpoint["epoch"]
# checkpoint_description = checkpoint["description"]
# print(checkpoint_description)

In [126]:
"""
정확도 측정
"""
label_list = []
predictions_list = []
total = 0
correct = 0
for x, y in test_dataloader:
    x = x.to(device)
    y = y.to(device)
    output = model(x)
    predictions = torch.max(output, 1)[1]
    predictions_list.append(predictions)
    correct += (predictions==y).sum()
    total +=len(y)
print(correct * 100 / total)

tensor(100.)
